In [1]:
import os
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
# ==============================
# 1. BASIC CONFIG
# ==============================
# Use the split dataset
DATA_DIR = r"C:\Harsh Works\code\American Sign Language\ASL_Split"

TRAIN_DIR = os.path.join(DATA_DIR, "train")
VAL_DIR = os.path.join(DATA_DIR, "val")
TEST_DIR = os.path.join(DATA_DIR, "test")

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
SEED = 42

# Output folder for models, plots, CSV
OUTPUT_DIR = r"C:\Harsh Works\code\American Sign Language\asl_training_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# ==============================
# 2. LOAD DATA FROM FOLDERS
# ==============================
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)
test_ds = test_ds.prefetch(AUTOTUNE)


# Create smaller training dataset for weaker Model 1
train_batches = tf.data.experimental.cardinality(train_ds).numpy()
baseline_fraction = 0.5  # try 0.3 or 0.2 if still perfect accuracy

limited_train_ds = train_ds.take(int(train_batches * baseline_fraction))

print("Total train batches:", train_batches)
print("Baseline train batches (Model 1):", int(train_batches * baseline_fraction))


Found 2516 files belonging to 6 classes.
Found 533 files belonging to 6 classes.
Found 533 files belonging to 6 classes.
Classes: ['Hello', 'I Love You', 'Okay', 'Please', 'Thank you', 'Yes']
Total train batches: 79
Baseline train batches (Model 1): 39


In [4]:
# ==============================
# 3. DATA AUGMENTATION LAYER
# ==============================
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ],
    name="data_augmentation",
)


In [5]:
# ==============================
# 4. MODEL 1 – SIMPLE CNN
# ==============================
def build_model_1(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=6):
    model = models.Sequential([
        layers.Rescaling(1./255, input_shape=input_shape),
        data_augmentation,

        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


In [6]:
# ==============================
# 5. MODEL 2 – TRANSFER LEARNING (MobileNetV2)
# ==============================
def build_model_2(input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3), num_classes=6):
    base_model = keras.applications.MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights="imagenet",
    )
    base_model.trainable = False  # freeze base layers

    inputs = keras.Input(shape=input_shape)
    x = layers.Rescaling(1./255)(inputs)
    x = data_augmentation(x)
    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model





In [7]:
# ==============================
# 6. TRAIN BOTH MODELS
# ==============================
EPOCHS = 15
# EPOCHS_MODEL1 = 5
# EPOCHS_MODEL2 = 15

# print("\n================= TRAINING MODEL 1 =================")
# model1 = build_model_1(num_classes=num_classes)
# history1 = model1.fit(
#     limited_train_ds,          # <-- weaker dataset
#     validation_data=val_ds,
#     epochs=EPOCHS_MODEL1,
# )

print("\n================= TRAINING MODEL 1: SIMPLE CNN =================")
model1 = build_model_1(num_classes=num_classes)
history1 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)
test_loss1, test_acc1 = model1.evaluate(test_ds)
print("Model 1 Test Accuracy:", test_acc1)

# SAVE MODEL 1
model1_path = os.path.join(OUTPUT_DIR, "asl_model_cnn.h5")
model1.save(model1_path)
print(f"✅ Model 1 saved as {model1_path}")


print("\n================= TRAINING MODEL 2: MobileNetV2 =================")
model2 = build_model_2(num_classes=num_classes)
history2 = model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
)
test_loss2, test_acc2 = model2.evaluate(test_ds)
print("Model 2 Test Accuracy:", test_acc2)

# SAVE MODEL 2
model2_path = os.path.join(OUTPUT_DIR, "asl_model_mobilenetv2.h5")
model2.save(model2_path)
print(f"✅ Model 2 saved as {model2_path}")


================= TRAINING MODEL 1: SIMPLE CNN =================
Epoch 1/15


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\preprocessing\data_layer.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


79/79 ━━━━━━━━━━━━━━━━━━━━ 18s 207ms/step - accuracy: 0.4960 - loss: 1.2811 - val_accuracy: 0.9212 - val_loss: 0.2575
Epoch 2/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 161ms/step - accuracy: 0.8776 - loss: 0.3670 - val_accuracy: 0.9606 - val_loss: 0.1047
Epoch 3/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 161ms/step - accuracy: 0.9086 - loss: 0.2661 - val_accuracy: 0.9869 - val_loss: 0.0370
Epoch 4/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 162ms/step - accuracy: 0.9304 - loss: 0.2050 - val_accuracy: 0.9831 - val_loss: 0.0373
Epoch 5/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 162ms/step - accuracy: 0.9507 - loss: 0.1498 - val_accuracy: 0.9981 - val_loss: 0.0097
Epoch 6/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 161ms/step - accuracy: 0.9519 - loss: 0.1481 - val_accuracy: 0.9794 - val_loss: 0.0399
Epoch 7/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 161ms/step - accuracy: 0.9567 - loss: 0.1273 - val_accuracy: 0.9981 - val_loss: 0.0074
Epoch 8/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 13s 163ms/step - accuracy: 0.9642 - loss: 0.1068 - val_accuracy: 1.000

Model 1 Test Accuracy: 1.0
✅ Model 1 saved as C:\Harsh Works\code\American Sign Language\asl_training_outputs\asl_model_cnn.h5

================= TRAINING MODEL 2: MobileNetV2 =================
Epoch 1/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 16s 167ms/step - accuracy: 0.6975 - loss: 0.8606 - val_accuracy: 0.9287 - val_loss: 0.2547
Epoch 2/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 12s 147ms/step - accuracy: 0.9173 - loss: 0.2415 - val_accuracy: 0.9700 - val_loss: 0.1396
Epoch 3/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 140ms/step - accuracy: 0.9567 - loss: 0.1467 - val_accuracy: 0.9812 - val_loss: 0.1044
Epoch 4/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 12s 151ms/step - accuracy: 0.9710 - loss: 0.1117 - val_accuracy: 0.9831 - val_loss: 0.0817
Epoch 5/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 142ms/step - accuracy: 0.9722 - loss: 0.0988 - val_accuracy: 0.9625 - val_loss: 0.1072
Epoch 6/15
79/79 ━━━━━━━━━━━━━━━━━━━━ 11s 137ms/step - accuracy: 0.9837 - loss: 0.0711 - val_accuracy: 0.9869 - val_loss: 0.0616
Epoch 7/15
79/79 ━━━━━━━━━━━━━━━

Model 2 Test Accuracy: 0.9774859547615051
✅ Model 2 saved as C:\Harsh Works\code\American Sign Language\asl_training_outputs\asl_model_mobilenetv2.h5


In [8]:
# ==============================
# 7. PREDICTIONS + METRICS
# ==============================
from sklearn.metrics import classification_report, confusion_matrix

# Helper to flatten dataset into arrays
def get_true_and_pred_labels(model, dataset):
    all_images = []
    all_labels = []

    for images, labels in dataset:
        all_images.append(images.numpy())
        all_labels.append(labels.numpy())

    all_images = np.concatenate(all_images, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    preds = model.predict(all_images, verbose=0)
    y_pred = np.argmax(preds, axis=1)

    return all_labels, y_pred

# Get labels/preds for both models on the TEST SET
y_true1, y_pred1 = get_true_and_pred_labels(model1, test_ds)
y_true2, y_pred2 = get_true_and_pred_labels(model2, test_ds)

# All possible class indices (0..num_classes-1)
all_labels_indices = list(range(num_classes))

print("\n================= MODEL 1 CLASSIFICATION REPORT =================")
print(
    classification_report(
        y_true1,
        y_pred1,
        labels=all_labels_indices,
        target_names=class_names,
        zero_division=0,   # handle any zero-support class
        digits=4,
    )
)

print("\n================= MODEL 2 CLASSIFICATION REPORT =================")
print(
    classification_report(
        y_true2,
        y_pred2,
        labels=all_labels_indices,
        target_names=class_names,
        zero_division=0,
        digits=4,
    )
)

# Macro scores for table
def get_macro_scores(y_true, y_pred):
    report = classification_report(
        y_true,
        y_pred,
        labels=all_labels_indices,
        target_names=class_names,
        output_dict=True,
        zero_division=0,
    )
    precision = report["macro avg"]["precision"]
    recall = report["macro avg"]["recall"]
    f1 = report["macro avg"]["f1-score"]
    return precision, recall, f1

prec1, rec1, f1_1 = get_macro_scores(y_true1, y_pred1)
prec2, rec2, f1_2 = get_macro_scores(y_true2, y_pred2)



================= MODEL 1 CLASSIFICATION REPORT =================
              precision    recall  f1-score   support

       Hello     1.0000    1.0000    1.0000        92
  I Love You     1.0000    1.0000    1.0000        87
        Okay     1.0000    1.0000    1.0000        72
      Please     1.0000    1.0000    1.0000       131
   Thank you     1.0000    1.0000    1.0000        73
         Yes     1.0000    1.0000    1.0000        78

    accuracy                         1.0000       533
   macro avg     1.0000    1.0000    1.0000       533
weighted avg     1.0000    1.0000    1.0000       533


================= MODEL 2 CLASSIFICATION REPORT =================
              precision    recall  f1-score   support

       Hello     1.0000    1.0000    1.0000        92
  I Love You     1.0000    1.0000    1.0000        87
        Okay     1.0000    1.0000    1.0000        72
      Please     0.9225    1.0000    0.9597       131
   Thank you     1.0000    0.8356    0.9104        7

In [9]:
# ==============================
# 8. COMPARISON TABLE (PANDAS DF)
# ==============================
results_df = pd.DataFrame({
    "Model": ["Simple CNN (Model 1)", "MobileNetV2 (Model 2)"],
    "Test Accuracy": [test_acc1, test_acc2],
    "Macro Precision": [prec1, prec2],
    "Macro Recall": [rec1, rec2],
    "Macro F1-Score": [f1_1, f1_2],
})

print("\n=== MODEL COMPARISON TABLE ===")
print(results_df)

csv_path = os.path.join(OUTPUT_DIR, "model_comparison_results.csv")
results_df.to_csv(csv_path, index=False)
print(f"📄 Results table saved to {csv_path}")



=== MODEL COMPARISON TABLE ===
                   Model  Test Accuracy  Macro Precision  Macro Recall  \
0   Simple CNN (Model 1)       1.000000         1.000000      1.000000   
1  MobileNetV2 (Model 2)       0.977486         0.984979      0.972603   

   Macro F1-Score  
0        1.000000  
1        0.977298  
📄 Results table saved to C:\Harsh Works\code\American Sign Language\asl_training_outputs\model_comparison_results.csv


In [10]:
# ==============================
# 9. TRAINING CURVE PLOTS
# ==============================
def plot_history(history, title_prefix, file_prefix):
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs_range = range(1, len(acc) + 1)

    # Accuracy
    plt.figure()
    plt.plot(epochs_range, acc, label="Train Acc")
    plt.plot(epochs_range, val_acc, label="Val Acc")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.title(f"{title_prefix} - Accuracy")
    plt.legend()
    acc_path = os.path.join(OUTPUT_DIR, f"{file_prefix}_accuracy.png")
    plt.savefig(acc_path)
    plt.close()

    # Loss
    plt.figure()
    plt.plot(epochs_range, loss, label="Train Loss")
    plt.plot(epochs_range, val_loss, label="Val Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"{title_prefix} - Loss")
    plt.legend()
    loss_path = os.path.join(OUTPUT_DIR, f"{file_prefix}_loss.png")
    plt.savefig(loss_path)
    plt.close()

    print(f"📊 {title_prefix} plots saved as:\n  {acc_path}\n  {loss_path}")

plot_history(history1, "Model 1 (Simple CNN)", "model1")
plot_history(history2, "Model 2 (MobileNetV2)", "model2")

📊 Model 1 (Simple CNN) plots saved as:
  C:\Harsh Works\code\American Sign Language\asl_training_outputs\model1_accuracy.png
  C:\Harsh Works\code\American Sign Language\asl_training_outputs\model1_loss.png
📊 Model 2 (MobileNetV2) plots saved as:
  C:\Harsh Works\code\American Sign Language\asl_training_outputs\model2_accuracy.png
  C:\Harsh Works\code\American Sign Language\asl_training_outputs\model2_loss.png


In [11]:
# ==============================
# 10. CONFUSION MATRIX PLOTS
# ==============================
def plot_confusion_matrix(cm, classes, title, filename,
                          normalize=False,
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(6, 6))
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, ha="right")
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel("Predicted label")
    plt.savefig(filename)
    plt.close()
    print(f"📌 Confusion matrix saved as {filename}")

cm1 = confusion_matrix(y_true1, y_pred1)
cm2 = confusion_matrix(y_true2, y_pred2)

cm1_path = os.path.join(OUTPUT_DIR, "model1_confusion.png")
cm2_path = os.path.join(OUTPUT_DIR, "model2_confusion.png")

plot_confusion_matrix(cm1, class_names, "Model 1 Confusion Matrix", cm1_path)
plot_confusion_matrix(cm2, class_names, "Model 2 Confusion Matrix", cm2_path)

📌 Confusion matrix saved as C:\Harsh Works\code\American Sign Language\asl_training_outputs\model1_confusion.png
📌 Confusion matrix saved as C:\Harsh Works\code\American Sign Language\asl_training_outputs\model2_confusion.png


In [12]:
# ==============================
# 11. BAR CHART: ACCURACY & F1
# ==============================
labels_models = ["Model 1", "Model 2"]
acc_values = [test_acc1, test_acc2]
f1_values = [f1_1, f1_2]

x = np.arange(len(labels_models))
width = 0.35

plt.figure()
plt.bar(x - width/2, acc_values, width, label="Accuracy")
plt.bar(x + width/2, f1_values, width, label="F1-Score")
plt.xticks(x, labels_models)
plt.ylabel("Score")
plt.ylim(0, 1.0)
plt.title("Final Test Metrics: Model 1 vs Model 2")
plt.legend()
bar_path = os.path.join(OUTPUT_DIR, "model_metrics_bar.png")
plt.savefig(bar_path)
plt.close()

print(f"📊 Bar chart saved as {bar_path}")
print("\n🎉 All done! Models, metrics, and plots are saved in:", OUTPUT_DIR)

📊 Bar chart saved as C:\Harsh Works\code\American Sign Language\asl_training_outputs\model_metrics_bar.png

🎉 All done! Models, metrics, and plots are saved in: C:\Harsh Works\code\American Sign Language\asl_training_outputs
